This notebook attempts to perform neighborhood segementation and clustering in the city of Toronto, Canada.

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

**Part 1**: Scrape Toronto Neighborhood from Wikipedia with Beautiful Soup

In [2]:
# PART 1
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source)
table = soup.find('table', attrs={'class':'wikitable sortable'})
table_body = table.find('tbody')

In [3]:
data = []
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    if cols == [] or cols[1] == "Not assigned":
        continue
    if cols[2] == "Not assigned":
        cols[2] = str(float("NaN"))
    data.append([ele for ele in cols if ele])
df = pd.DataFrame(data, columns=["Postalcode", "Borough", "Neighbourhood"])
df["PB"] = df['Postalcode'] + ', ' + df['Borough']
df = df.groupby('PB')['Neighbourhood'].apply(', '.join).reset_index()
df = pd.concat([df, df.PB.str.split(", ", expand=True)], axis = 1)
df.drop(["PB"], axis = 1, inplace = True)
df.columns = ["Neighbourhood", "Postalcode", "Borough"]
columnsTitles=["B","A"]
df=df.reindex(columns=["Postalcode", "Borough", "Neighbourhood"])
for i, s in enumerate(df["Neighbourhood"]):
    if s == str(float("NaN")):
        df["Neighbourhood"][i] = df["Borough"][i]
df = df.sort_values("Postalcode")
df

,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...


In [4]:
df.shape

(103, 3)

**Part 2**: Include the coordinates for each neighbourhood.

In [5]:
coords = pd.read_csv("Geospatial_Coordinates.csv").sort_values("Postal Code")
df = df.join(coords[["Latitude", "Longitude"]], how = "outer")
df

FileNotFoundError: [Errno 2] File Geospatial_Coordinates.csv does not exist: 'Geospatial_Coordinates.csv'